# 说明：

神经网络系数数值小的、各种数据其数值波动范围不大的情况！—— 需要“**标准化**”操作。数据标准化让机器学习的模型看到不同样本彼此之间更加相似，有助于模型的学习与对新数据的泛化。

标准化：1. 标准化；2. 归一化

批标准化：普通的标准化只是对“输入时的数据”进行标准化操作；但是当数据被“**有训练参数**”的层输出时，它们又不满足标准化的要求的了（均值、方差都变了）！因此批标准化，就是：对网络中“**每一次变化之后**”都进行数据标准化。

批标准化的好处：
- 加速收敛（只对输入数据做标准化都能加速收敛，何况是对每次变化都做）：因为数值变化范围小了；
- 具有正则化的效果；
- 提高模型的泛化能力；
- 允许更高的学习速率，从而加速收敛；
- 有助于梯度下降的传播（梯度值不易为0），因此允许更深的网络！

keras中内置的大型网络，都已经自带了批标准化层（批标准化以“层”的形式加入，很简单：layers.Batchnormalization() ）—— 文件29中的Xception的层中就有很多“批标准化层”。

# 在自己的网络中使用“批标准化”：

使用文件22中的自己搭的网络，对其加入批标准化：

In [1]:
import keras
from keras import layers
import numpy as np
import os
import shutil    # os和shutil用来处理文件

Using TensorFlow backend.


In [2]:
# 原始数据太多了，现在专门创建一个文件夹来存储一部分要用的训练集 + 测试集
base_dir = 'E:/Python_code/keras_total/日月光华-keras课程资料/dc/try'
train_dir = os.path.join(base_dir, 'train')
train_dir_dog = os.path.join(train_dir , 'dog')
train_dir_cat = os.path.join(train_dir , 'cat')

test_dir = os.path.join(base_dir , 'test')
test_dir_dog = os.path.join(test_dir , 'dog')
test_dir_cat = os.path.join(test_dir , 'cat')

dc_dir = 'E:/Python_code/keras_total/日月光华-keras课程资料/dc/train' # 原始数据所在路径

In [3]:
# 训练集猫狗各1000张，测试集猫狗各500张。
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
    os.mkdir(train_dir)
    os.mkdir(train_dir_dog)
    os.mkdir(train_dir_cat)
    os.mkdir(test_dir)
    os.mkdir(test_dir_dog)
    os.mkdir(test_dir_cat)

    fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname) 
        d = os.path.join(train_dir_cat, fname)
        shutil.copyfile(s, d)   #  把s拷贝到d 

    fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname)
        d = os.path.join(test_dir_cat, fname)
        shutil.copyfile(s, d)

    fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname)
        d = os.path.join(train_dir_dog, fname)
        shutil.copyfile(s, d)

    fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
    for fname in fnames:
        s = os.path.join(dc_dir, fname)
        d = os.path.join(test_dir_dog, fname)
        shutil.copyfile(s, d)

In [4]:
from keras.preprocessing.image import ImageDataGenerator

In [5]:
# 创建图片的迭代器，并且设定它的归一化
train_datagen = ImageDataGenerator(1/255)
test_datagen = ImageDataGenerator(1/255)

In [6]:
# 正式创建图片的生成器：train
train_generator = train_datagen.flow_from_directory( train_dir,                  # 待读取文件的目录
                                                     target_size = (200,200),    # 图片的统一大小 
                                                     batch_size = 20,            # 每次读入20张
                                                     class_mode = 'binary'       # 该文件夹下分两类：因为我已经正好在该文件夹下分了两个文件夹
)

# 正式创建图片的生成器：test
test_generator = test_datagen.flow_from_directory( test_dir,                  # 待读取文件的目录
                                                    target_size = (200,200),    # 图片的统一大小 
                                                    batch_size = 20,            # 每次读入20张
                                                    class_mode = 'binary'       # 该文件夹下分两类：因为我已经正好在该文件夹下分了两个文件夹
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# 模型搭建：加入“批标准化层”

In [7]:
model = keras.Sequential()

In [8]:
from keras import layers

In [9]:
# 在每一个“有训练参数”的层后加入“批标准化层” —— 卷积层 + Dense层 之后
model.add( layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(200,200,3)) )
model.add( layers.BatchNormalization() )  # 批标准化层
model.add( layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu') )
model.add( layers.BatchNormalization() )  # 批标准化层

model.add( layers.MaxPooling2D() )
model.add( layers.Dropout(0.25) )  

model.add( layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu') )
model.add( layers.BatchNormalization() )  # 批标准化层
model.add( layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu') )
model.add( layers.BatchNormalization() )  # 批标准化层

model.add( layers.MaxPooling2D() )
model.add( layers.Dropout(0.25) )  # 防止过拟合

model.add( layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu') )
model.add( layers.BatchNormalization() )  # 批标准化层
model.add( layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu') )
model.add( layers.BatchNormalization() )  # 批标准化层

model.add( layers.MaxPooling2D() )
model.add( layers.Dropout(0.25) )  # 防止过拟合

# 到了全连接层：
model.add( layers.Flatten() )

model.add( layers.Dense(256, activation='relu') )
model.add( layers.BatchNormalization() )  # 批标准化层

model.add( layers.Dropout(0.5) )
model.add( layers.Dense(1, activation='sigmoid') )   # 二分类，输出层用sigmoid

# 有了批标准化层后，可以增大层的深度！

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 198, 198, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 198, 198, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 196, 196, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 196, 196, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 98, 98, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 98, 98, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 96, 96, 64)       

In [11]:
model.compile( optimizer='adam',
               loss = 'binary_crossentropy',
               metrics=['acc']
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
# 此时输入数据是“generator生成器”，所以用model.fit_generator
history = model.fit_generator( 
     # 训练数据：
     train_generator,
     epochs = 3,
     steps_per_epoch = 100,  # 有2000张图，每次录入20张，因此100步可以走完一个epoch！不设置的话，生成器会一直循环生成
     # 测试数据：
     validation_data = test_generator, 
     validation_steps = 50   # 有1000张图，每次录入20张，故50步即可走完一个epoch！
)